In [ ]:
!pip install "transformers==2.11.0"
!pip install pyarabic
!pip install farasapy
!pip install pyarabic
!git clone https://github.com/aub-mind/arabert

     |████████████████████████████████| 675kB 13.1MB/s 
     |████████████████████████████████| 890kB 44.8MB/s 
     |████████████████████████████████| 3.8MB 53.8MB/s 
     |████████████████████████████████| 1.1MB 56.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=5b1329bc9241253e4d1334d2d911abb0405b37bf97b962c82a931ad84b40f00f
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 112kB 22.7MB/s 
  Created wheel for pyarabic: filename=PyArabic-0.6.10-cp36-none-any.whl size=113324 sha256=c3a4d979a7296d2cd2a6732553e726a013e26f1f48a1e0ae11edd9df9d219ad3
  Stored in directory: /root/.cache/pip/wheels/10/b8/f5/b7c1a50e6efb83544844f165a9b134afe7292585465e29b61d
Successfully built pyarabic
Cloning into 'arabert'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing

In [ ]:
import pandas as pd
import numpy as np

from farasa.segmenter import FarasaSegmenter
from arabert.preprocess_arabert import preprocess, never_split_tokens
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, precision_score , recall_score

from transformers import AutoConfig, BertForSequenceClassification, AutoTokenizer, AutoModel
from transformers.data.processors import SingleSentenceClassificationProcessor
from transformers import Trainer , TrainingArguments

import logging
import pandas as p
import re
import string

farasa_segmenter = FarasaSegmenter(interactive=True)

  0%|          | 0.00/241M [00:00<?, ?iB/s]

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


100%|██████████| 241M/241M [00:08<00:00, 28.8MiB/s]


[2020-12-26 16:00:42,804 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

DATA_COLUMN = 'line'
LABEL_COLUMN = 'dialect'
train_files = ["/content/drive/MyDrive/trainset/AE_targe_preprocessed.txt", "/content/drive/MyDrive/trainset/BH_targe_preprocessed.txt","/content/drive/MyDrive/trainset/DZ_targe_preprocessed.txt",
               "/content/drive/MyDrive/trainset/EG_targe_preprocessed.txt", "/content/drive/MyDrive/trainset/IQ_targe_preprocessed.txt","/content/drive/MyDrive/trainset/JO_targe_preprocessed.txt",
               "/content/drive/MyDrive/trainset/KW_targe_preprocessed.txt", "/content/drive/MyDrive/trainset/LB_targe_preprocessed.txt","/content/drive/MyDrive/trainset/LY_targe_preprocessed.txt",
               "/content/drive/MyDrive/trainset/QA_targe_preprocessed.txt", "/content/drive/MyDrive/trainset/OM_targe_preprocessed.txt","/content/drive/MyDrive/trainset/PL_targe_preprocessed.txt",
               "/content/drive/MyDrive/trainset/OM_targe_preprocessed.txt", "/content/drive/MyDrive/trainset/SA_targe_preprocessed.txt","/content/drive/MyDrive/trainset/SD_targe_preprocessed.txt",
               "/content/drive/MyDrive/trainset/SY_targe_preprocessed.txt", "/content/drive/MyDrive/trainset/TN_targe_preprocessed.txt","/content/drive/MyDrive/trainset/YE_targe_preprocessed.txt"]

country_names = ["AE","BH","DZ","EG","IQ","JO","KW","LB","LY","MA","OM","PL","QA","SA","SD","SY","TN","YE"]

label_map = {
    'AE' : 0,
    'BH' : 1,
    'DZ' : 2,
    'EG' : 3,
    'IQ' : 4,
    'JO' : 5,
    'KW' : 6,
    'LB' : 7,
    'LY' : 8,
    'MA' : 9,
    'OM' : 10,
    'PL' : 11,
    'QA' : 12,
    'SA' : 13,
    'SD' : 14,
    'SY' : 15,
    'TN' : 16,
    'YE' : 17
}

print(train_files)
train_data_list = []
for index in range(len(train_files)):
  train_data_list.append(p.read_csv(train_files[index], lineterminator= '~', header=None, names = ['line']))
  train_data_list[index][LABEL_COLUMN] = [country_names[index] for i in range(len(train_data_list[index].index))]

train_data = p.concat(train_data_list)

# test_data = p.read_csv('/content/drive/MyDrive/testset/QADI_test.txt', sep = '\t', header=None, lineterminator='\n')
# test_data.columns = [DATA_COLUMN, LABEL_COLUMN]


# train_data = p.read_csv('/content/drive/MyDrive/data/train_concatinated.txt', sep = '\t', header=None)

Mounted at /content/drive
['/content/drive/MyDrive/trainset/AE_targe_preprocessed.txt', '/content/drive/MyDrive/trainset/BH_targe_preprocessed.txt', '/content/drive/MyDrive/trainset/DZ_targe_preprocessed.txt', '/content/drive/MyDrive/trainset/EG_targe_preprocessed.txt', '/content/drive/MyDrive/trainset/IQ_targe_preprocessed.txt', '/content/drive/MyDrive/trainset/JO_targe_preprocessed.txt', '/content/drive/MyDrive/trainset/KW_targe_preprocessed.txt', '/content/drive/MyDrive/trainset/LB_targe_preprocessed.txt', '/content/drive/MyDrive/trainset/LY_targe_preprocessed.txt', '/content/drive/MyDrive/trainset/QA_targe_preprocessed.txt', '/content/drive/MyDrive/trainset/OM_targe_preprocessed.txt', '/content/drive/MyDrive/trainset/PL_targe_preprocessed.txt', '/content/drive/MyDrive/trainset/OM_targe_preprocessed.txt', '/content/drive/MyDrive/trainset/SA_targe_preprocessed.txt', '/content/drive/MyDrive/trainset/SD_targe_preprocessed.txt', '/content/drive/MyDrive/trainset/SY_targe_preprocessed.txt

In [ ]:
train_data[DATA_COLUMN] = train_data[DATA_COLUMN].apply(lambda x: preprocess(x, do_farasa_tokenization=True , farasa=farasa_segmenter, use_farasapy = True))

train_data[LABEL_COLUMN] = train_data[LABEL_COLUMN].apply(lambda x: label_map[x])



In [ ]:
train_data

,line,dialect
0,[مستخدم] ياخي ال+ مدرب أختار +ه و+ ال+ مدرب دخ...,0
1,شو اللي قاعد يجري ف+ ال+ نصر يا أخو +ه . خسر ا...,0
2,اللي يبحث عن مشكل +ة ال+ وصل راح يحصل +ها ف+ ا...,0
3,[مستخدم] أنا مش معترض على تغيير عامر . اللي دخ...,0
4,تراجع مخيف في مستوى ال+ حارس ال+ كبير ماجد ناص...,0
...,...,...
9934,ال+ يوم ال+ طايف و+ بكر +ه مابعد بعدالطايف ب+ ...,17
9935,[مستخدم] [مستخدم] [مستخدم] [مستخدم] [مستخدم] [...,17
9936,و+ كمان يسمو +ه خادم ال+ حرم +ين ل+ عنت +ي علي...,17
9937,[مستخدم] كان ودي اصدق ك+ بس جب +ت +ها قوي +ه ق...,17


In [ ]:
train_set, test_set = train_test_split(train_data, test_size=0.025,random_state=42)

train_df = pd.DataFrame({
    'id':range(len(train_set)),
    'label':train_set[LABEL_COLUMN],
    'text': train_set[DATA_COLUMN].replace(r'\n', ' ', regex=True)
})

dev_df = pd.DataFrame({
    'id':range(len(test_set)),
    'label':test_set[LABEL_COLUMN],
    'text': test_set[DATA_COLUMN].replace(r'\n', ' ', regex=True)
})

In [ ]:
train_df

,id,label,text
12408,0,15,[مستخدم] يعني نحن +ا كان هنيئ +ا ل+ +نا ب+ حاف...
10269,1,16,هذا ما كتب +ه إدواردو غاليانو عن ال+ بوعزيزي URL
8823,2,14,أنا ما شفت ال+ حاج +ة دي من بدري لي +ه URL
15441,3,15,اذ +ا صار +ت مصيب +ه في ال+ بيت من ال+ متهم ال...
47106,4,3,[مستخدم] ال+ بنوك ال+ استثماري +ة أرباح +هم اك...
...,...,...,...
4797,477277,8,:هادية و+ تعرض +ت +ي ل+ ال+ أحراج في منشن ب+ سببي
21103,477278,11,[مستخدم] أنشهد و+ ال+ نعم في +ه و+ في ك+ يابو ...
2588,477279,4,[مستخدم] يصير عند ك+ سوشيل لايف تخبل و+ تروح ك...
1032,477280,5,شفي +هم ال+ نشامي ! و+ شباب معان قاعد +ين عند ...


In [ ]:
model_name = 'asafaya/bert-medium-arabic'
num_labels = 18
task_name = 'classification'
max_length = 128

In [ ]:
config = AutoConfig.from_pretrained(model_name,num_labels=num_labels, output_attentions=True) ##needed for the visualizations
tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=False, do_basic_tokenize=True)
model = BertForSequenceClassification.from_pretrained(model_name,config=config)
# tokenizer = AutoTokenizer.from_pretrained(model_name, do_basic_tokenize=True)
# model = AutoModel.from_pretrained(model_name)

In [ ]:
print(config)
print(model)

BertConfig {
  "_num_labels": 2,
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 512,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17"
  },
  "initializer_range": 0.02,
  "intermediate_size": 2048,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_10": 10,
    "LABEL_11": 11,
    "LABEL_12": 12,
    "LABEL_13": 13,
    "LABEL_14": 14,
    "LABEL_15": 15,
    "LABEL_16": 16,
    "LABEL_17": 17,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7,
 

In [ ]:
# converting the examples in the dataset into features to be fed into the model.
train_dataset = SingleSentenceClassificationProcessor(mode='classification')
test_dataset = SingleSentenceClassificationProcessor(mode='classification')

In [ ]:
train_dataset.add_examples(texts_or_text_and_labels=train_df['text'],labels=train_df['label'],overwrite_examples = True)
test_dataset.add_examples(texts_or_text_and_labels=dev_df['text'],labels=dev_df['label'],overwrite_examples = True)

print(train_dataset.examples[0])

InputExample(guid=None, text_a='[مستخدم] يعني نحن +ا كان هنيئ +ا ل+ +نا ب+ حافظ ال+ عرص', text_b=None, label=15)


In [ ]:
train_features = train_dataset.get_features(tokenizer = tokenizer, max_length = max_length)
test_features = test_dataset.get_features(tokenizer = tokenizer, max_length = max_length)



In [ ]:
len(train_features[0].input_ids)
print(train_features[0])
print(len(train_features))
print(len(test_features))
pos_train_labels = np.sum([feature.label for feature in train_features])
pos_test_labels = np.sum([feature.label for feature in test_features])

print(pos_train_labels)
print(pos_test_labels)

InputFeatures(input_ids=[2, 37, 8292, 39, 3503, 3010, 15, 223, 1841, 1, 15, 223, 247, 15, 15, 5500, 224, 15, 9412, 1716, 15, 16167, 1027, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=None, label=15)
477282
12239
3559042
91092


In [ ]:
training_args = TrainingArguments("./train")

training_args.do_train = True
training_args.output_dir = './results'
training_args.evaluate_during_training = True
training_args.adam_epsilon = 1e-8
training_args.learning_rate = 2e-5
training_args.warmup_steps = 0
training_args.per_gpu_train_batch_size = 16
training_args.per_gpu_eval_batch_size = 16
training_args.num_train_epochs= 3
training_args.logging_steps = (len(train_features) - 1) // training_args.per_gpu_train_batch_size + 1
training_args.save_steps = training_args.logging_steps
training_args.seed = 34
training_args.logging_dir = './logs'
# training_args.logging_dir = "gs://" from torch.utils.tensorboard import SummaryWriter supports google cloud storage

In [ ]:
print(training_args.logging_steps)

29831


In [ ]:
#https://github.com/huggingface/transformers/blob/master/src/transformers/trainer_utils.py
def compute_metrics(p): #p should be of type EvalPrediction
  preds = np.argmax(p.predictions, axis=1)
  assert len(preds) == len(p.label_ids)
  print(classification_report(p.label_ids,preds))
  print(confusion_matrix(p.label_ids,preds))

  # f1_Positive = f1_score(p.label_ids,preds,pos_label=1,average='binary')
  # f1_Negative = f1_score(p.label_ids,preds,pos_label=0,average='binary')
  macro_f1 = f1_score(p.label_ids,preds,average='macro')
  macro_precision = precision_score(p.label_ids,preds,average='macro')
  macro_recall = recall_score(p.label_ids,preds,average='macro')
  acc = accuracy_score(p.label_ids,preds)
  return {
      'macro_f1' : macro_f1,
      'macro_precision': macro_precision,
      'macro_recall': macro_recall,
      'accuracy': acc
  }

In [ ]:
trainer = Trainer(model=model,
                  args = training_args,
                  train_dataset = train_features,
                  eval_dataset = test_features,
                  compute_metrics = compute_metrics)

In [ ]:
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


Using deprecated `--per_gpu_eval_batch_size` argument which will be removed in a future version. Using `--per_device_eval_batch_size` is preferred.


{"loss": 1.77547564479277, "learning_rate": 1.3333333333333333e-05, "epoch": 1.0, "step": 29831}



              precision    recall  f1-score   support

           0       0.38      0.35      0.36       681
           1       0.35      0.25      0.29       671
           2       0.64      0.51      0.57       424
           3       0.69      0.84      0.76      1454
           4       0.64      0.52      0.57       419
           5       0.44      0.36      0.40       752
           6       0.40      0.62      0.49      1192
           7       0.62      0.70      0.66       853
           8       0.49      0.73      0.58       855
           9       0.50      0.48      0.49       834
          10       0.22      0.11      0.15       489
          11       0.50      0.52      0.51      1043
          12       0.31      0.24      0.27       502
          13       0.45      0.40      0.43       742
          14       0.74      0.51      0.61       378
          15       0.45      0.32      0.37       463
          16       0.65      0.40      0.49       243
          17       0.30   

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


...


              precision    recall  f1-score   support

           0       0.45      0.35      0.39       681
           1       0.38      0.32      0.35       671
           2       0.63      0.58      0.60       424
           3       0.66      0.88      0.75      1454
           4       0.61      0.56      0.58       419
           5       0.51      0.31      0.38       752
           6       0.44      0.64      0.52      1192
           7       0.62      0.74      0.68       853
           8       0.63      0.67      0.65       855
           9       0.51      0.53      0.52       834
          10       0.29      0.08      0.13       489
          11       0.53      0.56      0.55      1043
          12       0.34      0.35      0.35       502
          13       0.44      0.49      0.46       742
          14       0.68      0.56      0.61       378
          15       0.49      0.38      0.43       463
          16       0.75      0.46      0.57       243
          17       0.35   

Using deprecated `--per_gpu_eval_batch_size` argument which will be removed in a future version. Using `--per_device_eval_batch_size` is preferred.


{"loss": 1.2784759517099595, "learning_rate": 0.0, "epoch": 3.0, "step": 89493}



              precision    recall  f1-score   support

           0       0.42      0.42      0.42       681
           1       0.41      0.34      0.37       671
           2       0.67      0.57      0.62       424
           3       0.72      0.86      0.78      1454
           4       0.62      0.57      0.60       419
           5       0.51      0.39      0.45       752
           6       0.49      0.61      0.54      1192
           7       0.67      0.72      0.69       853
           8       0.63      0.69      0.66       855
           9       0.52      0.53      0.52       834
          10       0.29      0.29      0.29       489
          11       0.55      0.59      0.57      1043
          12       0.30      0.21      0.25       502
          13       0.43      0.51      0.46       742
          14       0.73      0.56      0.63       378
          15       0.52      0.37      0.43       463
          16       0.70      0.49      0.57       243
          17       0.34   

TrainOutput(global_step=89493, training_loss=1.4980275467759627)

In [ ]:
trainer.save_model('/content/drive/MyDrive/nlp_models/model_mini_5_epochs')

KeyboardInterrupt: ignored

In [ ]:
trainer.train()

In [ ]:
trainer.save_model('/content/drive/MyDrive/nlp_models/model_mini_5_epochs')